# PLN I

## Desafio 2

Se utilizará de base para realizar el desafío la notebook planteada en clase.

Importamos las bilbiotecas que se utilizarán en todo el desafío

In [1]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
from pypdf import PdfReader
import re
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models.callbacks import CallbackAny2Vec

from sklearn.manifold import TSNE
import plotly.express as px
from pathlib import Path

## Creación de corpus, prepocesamiento y obtención del modelo

Tomaremos como corpus distintos libros obtenidos de https://www.textos.info.

In [2]:
def build_corpus(pdf_folder):
    base_dir = Path.cwd()
    pdf_dir  = base_dir / pdf_folder

    corpus_parts = []
    for pdf_path in pdf_dir.glob("*.pdf"):
        with pdf_path.open("rb") as fh:
            reader = PdfReader(fh)
            pdf_text = "".join(page.extract_text() or "" for page in reader.pages)
            corpus_parts.append(pdf_text)

    return "\n".join(corpus_parts)

full_text = build_corpus("textos")

Luego, tras haber observado el texto, se plantea una limpieza simple de caracteres indeseados, para luego proceder a la separación en oraciones y creación del dataframe con ellas.

In [3]:
clean_text = full_text.replace('\n', ' ').strip().replace('—', '').replace('«', '').replace('»', '')

def extract_sentences(texto):
    oraciones = re.split(r'(?<=[.!?])\s+', texto.strip())
    return [o.strip() for o in oraciones if o.strip()]

sentences = extract_sentences(clean_text)

df_sentences = pd.DataFrame(sentences, columns=['sentence'])

Con el texto limpio, realizamos la separación del texto plano en un array de las palabras conformantes.

In [4]:
sentence_tokens = []

for sentence in df_sentences["sentence"]:
    sentence_tokens.append(text_to_word_sequence(sentence))

Luego, tomando el código observado en clase, planteo una clase callback para poder luego instanciarla y tener mayor visibilidad de los resultados del entrenamiento.

In [5]:
class Callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.losses = []

    def on_epoch_end(self, model):
        # Usar get_latest_training_loss() correctamente
        current_loss = model.get_latest_training_loss()
        
        if self.epoch == 0:
            # Primera época: la pérdida acumulada es la pérdida de esta época
            epoch_loss = current_loss
            print(f'Loss after epoch {self.epoch}: {epoch_loss}')
        else:
            # Épocas posteriores: restar la pérdida acumulada anterior
            epoch_loss = current_loss - self.losses[-1]
            print(f'Loss after epoch {self.epoch}: {epoch_loss}')
        
        self.losses.append(current_loss)
        self.epoch += 1

A continuación, instancio el modelo Word2Vec utilizando el modelo skipgram.

In [6]:
w2v_model = Word2Vec(min_count=3,    
                     window=4,       
                     vector_size=200, 
                     negative=20,
                     workers=1,
                     sg=1)     

A partir de las oraciones previamente generados y haciendo uso del modelo w2v_model, generamos el vocabulario y observamos su longitud.

In [7]:
w2v_model.build_vocab(sentence_tokens)
print("Largo del vocabulario:", len(w2v_model.wv.index_to_key))

Largo del vocabulario: 10745


Finalmente, entrenamos el modelo generador de vectores

In [8]:
callback = Callback()
w2v_model.train(sentence_tokens,
                total_examples=w2v_model.corpus_count,
                epochs=100,
                compute_loss=True,
                callbacks=[callback])

Loss after epoch 0: 4813678.0
Loss after epoch 1: 3500440.5
Loss after epoch 2: 3337151.5
Loss after epoch 3: 3283053.0
Loss after epoch 4: 3205151.0
Loss after epoch 5: 3110570.0
Loss after epoch 6: 3065288.0
Loss after epoch 7: 3011238.0
Loss after epoch 8: 2969992.0
Loss after epoch 9: 2925176.0
Loss after epoch 10: 2933974.0
Loss after epoch 11: 2871592.0
Loss after epoch 12: 2800084.0
Loss after epoch 13: 2728364.0
Loss after epoch 14: 2676456.0
Loss after epoch 15: 2622104.0
Loss after epoch 16: 2575588.0
Loss after epoch 17: 2533960.0
Loss after epoch 18: 2511260.0
Loss after epoch 19: 2471528.0
Loss after epoch 20: 2449892.0
Loss after epoch 21: 2434528.0
Loss after epoch 22: 2311860.0
Loss after epoch 23: 602936.0
Loss after epoch 24: 597896.0
Loss after epoch 25: 592992.0
Loss after epoch 26: 585352.0
Loss after epoch 27: 584960.0
Loss after epoch 28: 583408.0
Loss after epoch 29: 575800.0
Loss after epoch 30: 573256.0
Loss after epoch 31: 571088.0
Loss after epoch 32: 566128

(31051065, 45717200)

In [9]:
print("Tamaño del vocabulario:", len(w2v_model.wv))
print("total_examples:", w2v_model.corpus_count)      # después de build_vocab
print("total_words:",   w2v_model.corpus_total_words) # idem

Tamaño del vocabulario: 10745
total_examples: 30496
total_words: 457172


## Test de analogías

Investigamos los resultados obtenidos para distintas palabras de forma tal que podamos tener una visión cualitativa de la representación semántica obtenida en los embeddings. Para distintas palabras obtenemos los 10 términos más cercanos y más lejanos.

Para **AMOR**

In [10]:
w2v_model.wv.most_similar(positive=["amor"], topn=10)

[('afrenta', 0.399025022983551),
 ('atribuirlo', 0.3874158263206482),
 ('avanzado', 0.37406113743782043),
 ('dios', 0.37073183059692383),
 ('amado', 0.3676939308643341),
 ('tranquilícese', 0.3587466776371002),
 ('alimento', 0.353460431098938),
 ('jurado', 0.34900417923927307),
 ('síntoma', 0.34878504276275635),
 ('¡alabado', 0.3475513160228729)]

In [11]:
w2v_model.wv.most_similar(negative=["amor"], topn=10)

[('encontró', 0.1185823604464531),
 ('ponen', 0.09690879285335541),
 ('netherfield', 0.08403855562210083),
 ('estuvo', 0.07715610414743423),
 ('vería', 0.07145251333713531),
 ('antiguos', 0.06357530504465103),
 ('¡juventud', 0.06144867092370987),
 ('preguntarle', 0.060278117656707764),
 ('vestido', 0.05960054323077202),
 ('señoras', 0.058236222714185715)]

Se observa que entre los más cercanos aparecen palabras con alta probabilidad de estar asociadas a afecto o relaciones (“amado”, “sentía”, “¡déjame”, “¡vete”), lo que indica que el modelo parece captura parte del campo semántico emocional. A pesar de ello, los valores de similitud son relativamente bajos y se presentan términos que no parecen conteneder una relación semántica fuerte.

Los terminos más lejanos no presentan ninguna relación directa con la palabra amor, por lo que puede considerar que es correcta la distancia obtenida.

Para **CASA**

In [12]:
w2v_model.wv.most_similar(positive=["casa"], topn=10)

[('potchinkoff', 0.47556817531585693),
 ('saville', 0.40614116191864014),
 ('parroquial', 0.3982153534889221),
 ('¡dorian', 0.39444833993911743),
 ('longbourn', 0.38412782549858093),
 ('núm', 0.3776700496673584),
 ('domicilio', 0.37437301874160767),
 ('row', 0.3730911612510681),
 ('habitación', 0.37281668186187744),
 ('rezando', 0.36897754669189453)]

In [13]:
w2v_model.wv.most_similar(negative=["casa"], topn=10)

[('¡pardiez', 0.14524158835411072),
 ('grave', 0.08806530386209488),
 ('terminar', 0.0766368880867958),
 ('mozo', 0.07024313509464264),
 ('conmovida', 0.06022975593805313),
 ('carece', 0.05976417288184166),
 ('llenó', 0.05616145208477974),
 ('entero', 0.05566224083304405),
 ('único', 0.054190292954444885),
 ('¡juventud', 0.05227229371666908)]

Se observa que los términos próximos apuntan a habitaciones o lugares físicos internos (“habitación”, “alcoba”, “cuartito”), lo que es semánticamente correcto. Asimismo, se observa que nombres propios de distintas novelas cuyo contexto está asociado a "casa".
- "Potchinkoff" es un personaje de "Crimen y Castigo", al cual se lo menciona 3 veces en la novela y en todas, la frase que lo hace es "Casa de Potchinkoff"
- "saville", al igual que con "Potchinkoff", es mencionado en reiteradas ocaciones en la novela "La vuelta al mundo en 80 días" es frases junto a la palabra "casa"
- "longbourn" es un pueblo nombrado en la novela "Orgullo y prejuicio". En la novela se observa que es nombrado frecuentemente junto a palabras como "vivia" o similares, las cuales podrían generar una cercanía semántica con "casa"

Adicionalmente, los valores de similud son mejores que los observados en "amor", resultado que se aprecia por el conjunto de palabras obtenido.

Las palabras lejanas no se observa ninguna relación directa con casa ni locaciones, por lo que se considera correcto.

Para **HIJA**

In [14]:
w2v_model.wv.most_similar(positive=["hija"], topn=10)

[('educada', 0.4036574065685272),
 ('viudo', 0.39133313298225403),
 ('colegio', 0.3758120536804199),
 ('¡madre', 0.3676969110965729),
 ('deseó', 0.3608778715133667),
 ('activa', 0.3595145344734192),
 ('¡rodia', 0.3590278625488281),
 ('¡hija', 0.35249465703964233),
 ('margaret', 0.3518632650375366),
 ('wickham', 0.35030096769332886)]

In [15]:
w2v_model.wv.most_similar(negative=["hija"], topn=10)

[('¡pardiez', 0.18651004135608673),
 ('buques', 0.15995362401008606),
 ('despertaron', 0.13004595041275024),
 ('animadas', 0.08714123070240021),
 ('amables', 0.08316412568092346),
 ('numerosos', 0.08046203851699829),
 ('permanecieron', 0.07821153849363327),
 ('continuaron', 0.0772058442234993),
 ('comprendió', 0.07690193504095078),
 ('vasos', 0.0756891593337059)]

Se observan palabras cercanas al rol familiar o la educación: “educada”, “colegio” y “compañera”. Esto muestra que el modelo relaciona hija con contexto familiar. Los nombres propios como "wickham" y "Potchinkoff" son personajes de las novelas que conforman el corpus, aunque se desconoce su conexión semántica con la palabra "hija" según el desarrollo de las novelas.

Las palabras lejanas no se observa ninguna relación directa con hija, por lo que se considera correcto.

Continuando con la investigación, planteamos operaciones entre distintos vectores para obtener un vector resultante, buscando de esa manera identificar si los vectores poseen un sentido claro a lo largo de sus distintas dimensiones. Para eso utilizamos la funcionalidad most_similar, configurando correctamente los términos positivos y negativos. 

In [16]:
def analogy(pos1, pos2, neg, topn=10):
    return w2v_model.wv.most_similar(positive=[pos1, pos2], negative=[neg], topn=topn)

Creada la function, observamos distintos términos y evaluamos el resultado obtenido. 

In [17]:
terms_list = [
    ["padre","hija","madre"],
    ["cuerpo","boca","puerta"], 
    ["cama","viajar","dormir"]
]

for term in terms_list:
    print(analogy(term[0],term[1],term[2]))

[('colegio', 0.3525211215019226), ('empleado', 0.34867748618125916), ('educada', 0.30117475986480713), ('obsequio', 0.29565364122390747), ('despedido', 0.29350218176841736), ('activa', 0.2933144271373749), ('advierte', 0.2930837869644165), ('viuda', 0.29054850339889526), ('reprochable', 0.29054635763168335), ('poderes', 0.29033350944519043)]
[('húmedo', 0.32737496495246887), ('nervio', 0.30895280838012695), ('echaban', 0.3061980605125427), ('tierna', 0.2806163430213928), ('moreno', 0.28059956431388855), ('hundida', 0.2779492437839508), ('telón', 0.2745162546634674), ('pareces', 0.26990872621536255), ('independiente', 0.2678249776363373), ('interminables', 0.2675785720348358)]
[('da', 0.3343045115470886), ('herido', 0.3101859390735626), ('¡vamos', 0.3064868748188019), ('levantarse', 0.3057457506656647), ('delito', 0.3014605939388275), ('unidos', 0.30112603306770325), ('dedicarse', 0.299754798412323), ('anonadado', 0.2961161136627197), ('¿iba', 0.29505404829978943), ('aprieto', 0.2914158

**Conclusión:**

A partir de los resultados observados, podemos mencionar que el modelo actual sí refleja algunos resultados significativos, pero no cuenta con suficiente información para sostener analogías algebraicas fiables.

Es posible que el tamaño del embedding elegido (200) o el tamaño del corpus hayan limitado la capacidad de poder plantear emebeddings que contengan en sus dimensiones un representación semántica tal que sea posible generar asociaciones confiables a partir de operaciones algebraicas entre ellos.

## Observación mediante reducción de la dimensionalidad

Para poder ver gráficamente las relaciones entre los ditintos términos que componente el vocabulario, plateo un reducción de la dimensionalidad a partir del método t-SNE

Primero, planteamos la function para poder plantear visualizaciones en 2D y 3D respectivamente, basándome en la function vista en clase.

In [18]:
def reduce_dimensionality(model, dimensions = 2 ):
    wv_vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=dimensions, random_state=42)
    wv_vectors = tsne.fit_transform(wv_vectors)
    return wv_vectors, labels

Para **2D**

In [19]:
vecs, labels = reduce_dimensionality(w2v_model)

MAX_WORDS_2D=200
fig = px.scatter(x=vecs[:MAX_WORDS_2D,0], y=vecs[:MAX_WORDS_2D,1], text=labels[:MAX_WORDS_2D])
fig.show()

Luego, para **3D**

In [20]:
vecs, labels = reduce_dimensionality(w2v_model,3)

MAX_WORDS_3D = 100
fig = px.scatter_3d(x=vecs[:MAX_WORDS_3D,0], y=vecs[:MAX_WORDS_3D,1], z=vecs[:MAX_WORDS_3D,2],text=labels[:MAX_WORDS_3D])
fig.update_traces(marker_size = 2)

**Conclusiones:**

A partir de los graficos 2D y 3D podemos observar que:

**Gráfico 2D**

- La mayoría de las palabras se agrupan en una región muy chica del espacio, lo que indica que los vectores generados comparten direcciones similares. Esto sugiere que el modelo no logra diferenciar claramente los contextos semánticos entre gran parte del vocabulario.

- Se observan algunas palabras aisladas del núcleo, como "habitación", "puerta" u "ojos", que probablemente aparecen en contextos más específicos dentro del corpus.

- Aparecen también nombres propios como "gregorio", "elizabeth" o "razumikin", posicionados más alejados del conjunto principal, lo que puede explicarse por sus apariciones recurrentes pero en contextos narrativos muy particulares.


**Gráfico 3D**

- El conjunto principal de palabras se concentra en un volumen reducido cerca del origen, lo que refleja una baja dispersión en el espacio vectorial y, por tanto, poca diferenciación entre los significados representados.

- Algunas palabras frecuentes como "la", "no" o "por", aparecen completamente separadas del resto, lo cual puede deberse a que su alta frecuencia en el corpus.

- La forma general de la nube evidencia una asimetría, con algunos puntos alejados del centro, lo que indica que ciertas palabras presentan dimensiones latentes que el modelo logró distinguir parcialmente.